Setup

In [1]:
import torch
print(torch.__version__)

1.12.1+cu102


In [2]:
!export PATH=/usr/local/cuda-11.1/bin:$PATH

In [3]:
!pip uninstall torch-scatter --yes 
!pip uninstall torch-sparse --yes
!pip uninstall torch-cluster --yes
!pip uninstall torch-spline-conv --yes
!pip uninstall torch-geometric --yes
!pip uninstall glycowork --y

Found existing installation: torch-scatter 2.0.9
Uninstalling torch-scatter-2.0.9:
  Successfully uninstalled torch-scatter-2.0.9
Found existing installation: torch-sparse 0.6.15
Uninstalling torch-sparse-0.6.15:
  Successfully uninstalled torch-sparse-0.6.15
Found existing installation: torch-cluster 1.6.0
Uninstalling torch-cluster-1.6.0:
  Successfully uninstalled torch-cluster-1.6.0
Found existing installation: torch-spline-conv 1.2.1
Uninstalling torch-spline-conv-1.2.1:
  Successfully uninstalled torch-spline-conv-1.2.1
Found existing installation: torch-geometric 2.1.0.post1
Uninstalling torch-geometric-2.1.0.post1:
  Successfully uninstalled torch-geometric-2.1.0.post1
Found existing installation: glycowork 0.5.0
Uninstalling glycowork-0.5.0:
  Successfully uninstalled glycowork-0.5.0


In [4]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu102.html
!pip install torch-sparse --no-cache-dir -f https://data.pyg.org/whl/torch-1.12.0+cu102.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.12.0+cu102.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.12.0+cu102.html
!pip install torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu102.html
!pip install glycowork

Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu102.html
  Using cached https://data.pyg.org/whl/torch-1.12.0%2Bcu102/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl (8.0 MB)
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu102.html
     |████████████████████████████████| 2.9 MB 1.7 MB/s            
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu102.html
  Using cached https://data.pyg.org/whl/torch-1.12.0%2Bcu102/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl (1.4 MB)
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu102.html
  Using cached https://data.pyg.org/whl/torch-1.12.0%2Bcu102/torch_spline_conv-1.2.1-cp37-cp37m-linux_x86_64.whl (633 kB)
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu102.html
  Using cached torch_geometric-2.1.0.post1-py3-none-any.whl
  Using cached glycowork-0.5.0-py3-none-any.whl (185.8 MB)


In [2]:
import glycowork
#export
from glycowork.ml.model_training import *
from glycowork.ml.models import *
from glycowork.ml.processing import *
from glycowork.ml.inference import *
from glycowork.ml.train_test_split import *
from glycowork.ml import models
from glycowork.ml import model_training
#export
from glycowork.glycan_data.loader import *
from glycowork.glycan_data.data_entry import *
from glycowork.motif.analysis import plot_embeddings, make_heatmap, characterize_monosaccharide, get_pvals_motifs

Parse Protein Input

In [19]:
!gsutil ls gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork
file2 = '10gb.txt'#temp name, change later
!gsutil cp gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/10gb.txt 10gb.txt
protein_db = []
with open('10gb.txt', 'r') as file2:
    for line in (file2):
        protein_db.append((line.rstrip()))
        print((line.rstrip()))
protein_embedding = []

gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/
gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/10gb.txt
gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/stored_protein_embeddings.pkl
Copying gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/10gb.txt...
/ [1 files][  106.0 B/  106.0 B]                                                
Operation completed over 1 objects/106.0 B.                                      
RLALEFRALETEGLLL
YPSVSFNLTGADTYEPFLRALQ
ARQQAEEASQESENEL
AALHFNPRLDTSEVVFNSKEGGSWG
LTSLVPVEPGRWHRLELSR


In [ ]:
import os
bucket_root = os.environ['WORKSPACE_BUCKET']
print(bucket_root)
# find out the path to file by running gsutil commandline tool.
!gsutil ls gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork
file1 = 'stored_protein_embeddings.pkl'

!gsutil cp gs://fc-0a6f0af0-7c38-45d2-9faa-2ab086ec0b09/Glycowork/stored_protein_embeddings.pkl stored_protein_embeddings.pkl

with open('stored_protein_embeddings.pkl', 'rb') as file1:
  old_dic = pickle.load(file)

Input for Embeddings

In [6]:
!pip install nbdev
#hide
#default_exp ml
import warnings
warnings.filterwarnings("ignore")
from nbdev.showdoc import show_doc
from IPython.display import HTML
%load_ext autoreload
%autoreload 2

In [17]:
!pip install fair-esm
import esm
model_esm, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()

In [ ]:
#show_doc(get_esm1b_representations)

In [ ]:
#represent = get_esm1b_representations(lane, model_esm, alphabet)

In [20]:
for protein_sequence in protein_db:
    protein_embedding.append(get_esm1b_representations(protein_sequence, model_esm, alphabet))

Code

In [ ]:
agr = 'PSVPAFKGHSFLAFPTLRAYHTLRLALEFRALETEGLLLYNGNARGKDFLALALLDGHVQFRFDTGSGPAVLTSLVPVEPGRWHRLELSRHWRQGTLSVDGEAPVVGESPSGTDGLNLDTKLYVGGLPEEQVATVLDRTSVGIGLKGCIRMLDINNQQLELSDWQRAVVQSSGVGEC'
# Lectin Oracle Paper's proteins
protein_SNA = 'MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADTYEPFLRALQEKVILGNHTAFDLPVLNPESQVSDSNRFVLVPLTNPSGDTVTLAIDVVNLYVVAFSSNGKSYFFSGSTAVQRDNLFVDTTQEELNFTGNYTSLERQVGFGRVYIPLGPKSLDQAISSLRTYTLTAGDTKPLARGLLVVIQMVSEAARFRYIELRIRTSITDASEFTPDLLMLSMENNWSSMSSEIQQAQPGGIFAGVVQLRDERNNSIEVTNFRRLFELTYIAVLLYGCAPVTSSSYSNNAIDAQIIKMPVFRGGEYEKVCSVVEVTRRISGWDGLCVDVRYGHYIDGNPVQLRPCGNECNQLWTFRTDGTIRWLGKCLTASSSVMIYDCNTVPPEATKWVVSIDGTITNPHSGLVLTAPQAAEGTALSLENNIHAARQGWTVGDVEPLVTFIVGYKQMCLRENGENNFVWLEDCVLNRVQQEWALYGDGTIRVNSNRSLCVTSEDHEPSDLIVILKCEGSGNQRWVFNTNGTISNPNAKLLMDVAQRDVSLRKIILYRPTGNPNQQWITTTHPA'
gal_7 = 'MSNVPHKSSLPEGIRPGTVLRIRGLVPPNASRFHVNLLCGEEGGSDAALHFNPRLDTSEVVFNSKEGGSWGREERGPGVPFGRGQPFEVLIIASDDGFKAVVGDAQYHHFRHRLPLARVRLVEVGGDVQLDSVRIF'
lox_1 = 'DGMQLSQVSDLLTQEQANLTHQKKKLEGQISARQQAEEASQESENELKEMIETLARKLNEKSKEQMELHHQNLNLQETLKRVANCSAPCPQDWIWHGENCYLFSSGSFNWEKSQEKCLSLDAKLLKINSTADLDFIQQAISYSSFPFWMGLSRRNPSYPWLWEDGSPLMPHLFRVRGAVSQTYPSGTCAYIQRGAVYAENCILAAFSICQKKANLRAQVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYNSTYRVVSVLTVLHQDWLNGKEYKCKVSNKALPAPIEKTISKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFFLYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK'
lane = 'TCFANAESGTYFDGTGFAKAVGGFKVGLDLLVEFEALELRGVQPVSC'

In [ ]:
#list of proteins to feed, 
protein_list = [protein_SNA, gal_7, lox_1, lane]
protein_names = ['sna','gal','lox_1','laneseq']
#possibly switch to zip (name, sequence)??
# protein_list = [(SNA, abc), (gal_7, def,), (lox_1, ghi)]
#glycan = ['1,5-Anhydro-ManNAc-ol(2-4)GlcNAc1N']

In [ ]:
model = prep_model('LectinOracle',1,trained=True)

In [ ]:
outprint_multi_protein = pd.DataFrame(columns = ['name', 'sequence', 'pred'])

In [ ]:
#lane sequence
#glycan = ['Glc(b1-3)Xyl(a1-3)Glc(b1-3)Xyl']
glycan = ['GlcA(b1-3)Xyl(a1-3)GlcA(b1-3)Xyl']

In [ ]:
show_doc(get_lectin_preds)

In [ ]:
#multiple protein, single glycan
a=0
#need to provide dictionary for protein esm 1 presentations. 
#doesnt work protein_naming = R"{protein}" 
for protein in protein_list:
  outprint_multi_protein.at[a, 'name'] = protein_names[a] # replace with the actual name when reading from a file
  outprint_multi_protein.at[a, 'sequence'] = protein
  outprint_multi_protein.at[a, 'pred'] = (get_lectin_preds(protein, glycan,model,old_dic).at[0,'pred'])
  a += 1
outprint_multi_protein.head()

In [ ]:
# single protein, multiple glycan
#preds = get_lectin_preds(protein, df_glycan.glycan.tolist(),model,old_dic)
preds = get_lectin_preds(lane, df_glycan.glycan.tolist(),model,represent)
print(preds.head())

In [ ]:
# single protein, single glycan
preds = get_lectin_preds(protein_SNA, glycan,model,old_dic)
print(preds.head())